In [1]:
# Cell 1: Load event table
import sys
sys.path.append('..')

from src.data_loaders.earnings import load_earnings_calendar
import pandas as pd

# Load events
events_df = load_earnings_calendar(
    start_year=2023,
    end_year=2023,
    min_volume_usd=5_000_000,
    save_path="../data/processed/event_table_sample.parquet"
)

print(f"Loaded {len(events_df)} events")
print(f"Unique stocks: {events_df['permno'].nunique()}")
events_df.head(10)

WRDS recommends setting up a .pgpass file.
pgpass file created at C:\Users\chosu\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Fetching earnings calendar from WRDS...
Filtering by volume threshold...
✓ Saved 10843 events to ../data/processed/event_table_sample.parquet
  Date range: 2023-01-03 00:00:00 to 2023-12-28 00:00:00
  Unique stocks: 2789
Loaded 10843 events
Unique stocks: 2789


,event_id,permno,gvkey,ticker,rdq,datadate,fyearq,fqtr
0,E0000,16710.0,031494,SGH,2023-01-03,2022-11-30,2023,1
1,E0001,65306.0,010840,UNF,2023-01-04,2022-11-30,2023,1
2,E0002,85040.0,064929,SLP,2023-01-04,2022-11-30,2023,1
3,E0003,12211.0,014954,LNN,2023-01-05,2022-11-30,2023,1
4,E0004,16284.0,027576,SMPL,2023-01-05,2022-11-30,2023,1
5,E0005,16431.0,028790,LW,2023-01-05,2022-11-30,2022,2
6,E0006,18898.0,035423,KRUS,2023-01-05,2022-11-30,2023,1
7,E0007,19502.0,011264,WBA,2023-01-05,2022-11-30,2023,1
8,E0008,19578.0,036788,DCT,2023-01-05,2022-11-30,2023,1
9,E0009,41452.0,005572,HELE,2023-01-05,2022-11-30,2022,3


In [4]:
# Cell 2: Test TAQ loading with connection reuse

import pandas as pd
import wrds
from src.features.order_flow import calculate_daily_ofi

# Open ONE connection
db = wrds.Connection(wrds_username='sungcho9')

# Find first valid event
for idx, event in events_df.iterrows():
    test_date = (pd.to_datetime(event['rdq']) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    
    # Skip weekends
    if pd.to_datetime(test_date).weekday() >= 5:
        continue
    
    print(f"Testing: {event['ticker']} on {test_date}")
    
    # Query directly with existing connection
    date_str = pd.to_datetime(test_date).strftime('%Y%m%d')
    year = pd.to_datetime(test_date).year
    
    query = f"""
    SELECT 
        time_m as timestamp,
        price,
        size
    FROM taqm_{year}.ctm_{date_str}
    WHERE sym_root = '{event['ticker']}'
        AND time_m BETWEEN '09:30:00' AND '16:00:00'
        AND price > 0
        AND size > 0
    ORDER BY time_m
    LIMIT 100000
    """
    
    try:
        trades = db.raw_sql(query)
        
        if len(trades) > 0:
            trades['timestamp'] = pd.to_datetime(test_date + ' ' + trades['timestamp'].astype(str))
            
            print(f"✓ Loaded {len(trades):,} trades")
            
            # Calculate OFI
            daily_ofi = calculate_daily_ofi(
                trades_df=trades,
                permno=int(event['permno']),
                date=test_date
            )
            
            print("\nDaily Features:")
            print(daily_ofi)
            
            selected_event = event
            selected_date = test_date
            break
            
    except Exception as e:
        print(f"  Error: {str(e)[:100]}...")
        continue

db.close()
print(f"\n✓ Done")

Loading library list...
Done
Testing: SGH on 2023-01-02
  Error: (psycopg2.errors.UndefinedTable) relation "taqm_2023.ctm_20230102" does not exist
LINE 6:     FROM t...
Testing: UNF on 2023-01-03
✓ Loaded 3,486 trades

Daily Features:
permno                          65306
date              2023-01-03 00:00:00
ofi                          0.015062
volume                          96267
num_trades                       3486
avg_trade_size              27.615318
buy_volume                      48842
sell_volume                     47392
dtype: object

✓ Done


In [5]:
# Cell 3: Calculate daily features
if len(trades) > 0:
    # Calculate OFI
    daily_ofi = calculate_daily_ofi(
        trades_df=trades,
        permno=event['permno'],
        date=test_date
    )
    
    # Calculate spread
    spread_metrics = calculate_daily_spread(quotes)
    
    # Combine
    daily_features = pd.concat([
        daily_ofi,
        pd.Series(spread_metrics)
    ])
    
    print("\nDaily Features:")
    print(daily_features)
    
    # Save to parquet
    daily_features_df = pd.DataFrame([daily_features])
    daily_features_df.to_parquet(
        "../data/processed/daily_features_sample.parquet",
        index=False
    )
    
    print("\n✓ Saved to daily_features_sample.parquet")


Daily Features:
permno                           65306.0
date                 2023-01-03 00:00:00
ofi                             0.015062
volume                             96267
num_trades                          3486
avg_trade_size                 27.615318
buy_volume                         48842
sell_volume                        47392
avg_spread                           NaN
avg_quoted_spread                    NaN
dtype: object

✓ Saved to daily_features_sample.parquet


In [6]:
daily_features_df = pd.read_parquet("../data/processed/daily_features_sample.parquet")
daily_features_df.describe()


,permno,date,ofi,volume,num_trades,avg_trade_size,buy_volume,sell_volume,avg_spread,avg_quoted_spread
count,1.0,1,1.000000,1.0,1.0,1.000000,1.0,1.0,0.0,0.0
mean,65306.0,2023-01-03 00:00:00,0.015062,96267.0,3486.0,27.615318,48842.0,47392.0,NaN,NaN
min,65306.0,2023-01-03 00:00:00,0.015062,96267.0,3486.0,27.615318,48842.0,47392.0,NaN,NaN
25%,65306.0,2023-01-03 00:00:00,0.015062,96267.0,3486.0,27.615318,48842.0,47392.0,NaN,NaN
50%,65306.0,2023-01-03 00:00:00,0.015062,96267.0,3486.0,27.615318,48842.0,47392.0,NaN,NaN
75%,65306.0,2023-01-03 00:00:00,0.015062,96267.0,3486.0,27.615318,48842.0,47392.0,NaN,NaN
max,65306.0,2023-01-03 00:00:00,0.015062,96267.0,3486.0,27.615318,48842.0,47392.0,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df = pd.read_parquet("../data/processed/daily_features_sample.parquet")
df.dtypes


permno                      float64
date                 datetime64[ns]
ofi                         float64
volume                        int64
num_trades                    int64
avg_trade_size              float64
buy_volume                    int64
sell_volume                   int64
avg_spread                  float64
avg_quoted_spread           float64
dtype: object